In [1]:
import win32com  

---
##### 전체 종목정보 가져오기
---

In [2]:
import win32com.client

instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
codeList = instCpCodeMgr.GetStockListByMarket(1)

for i, code in enumerate(codeList):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    print(i, code, secondCode, name)

0 A000020 1 동화약품
1 A000040 1 KR모터스
2 A000050 1 경방
3 A000070 1 삼양홀딩스
4 A000075 1 삼양홀딩스우
5 A000080 1 하이트진로
6 A000087 1 하이트진로2우B
7 A000100 1 유한양행
8 A000105 1 유한양행우
9 A000120 1 CJ대한통운
10 A000140 1 하이트진로홀딩스
11 A000145 1 하이트진로홀딩스우
12 A000150 1 두산
13 A000155 1 두산우
14 A000157 1 두산2우B
15 A000180 1 성창기업지주
16 A000210 1 DL
17 A000215 1 DL우
18 A000220 1 유유제약
19 A000225 1 유유제약1우
20 A000227 1 유유제약2우B
21 A000230 1 일동홀딩스
22 A000240 1 한국앤컴퍼니
23 A000270 1 기아
24 A000300 1 대유플러스
25 A000320 1 노루홀딩스
26 A000325 1 노루홀딩스우
27 A000370 1 한화손해보험
28 A000390 1 삼화페인트
29 A000400 1 롯데손해보험
30 A000430 1 대원강업
31 A000480 1 조선내화
32 A000490 1 대동
33 A000500 1 가온전선
34 A000520 1 삼일제약
35 A000540 1 흥국화재
36 A000545 1 흥국화재우
37 A000547 1 흥국화재2우B
38 A000590 1 CS홀딩스
39 A000640 1 동아쏘시오홀딩스
40 A000650 1 천일고속
41 A000660 1 SK하이닉스
42 A000670 1 영풍
43 A000680 1 LS네트웍스
44 A000700 1 유수홀딩스
45 A000720 1 현대건설
46 A000725 1 현대건설우
47 A000760 1 이화산업
48 A000810 1 삼성화재
49 A000815 1 삼성화재우
50 A000850 1 화천기공
51 A000860 1 강남제비스코
52 A000880 1 한화
53 A000885 1 

In [3]:
#종목정보 가져오기(최초)

import win32com.client
import pandas as pd 

rows = list()

CPE_MARKET_KIND = {'KOSPI':1,  'KOSDAQ':2}
instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")

for key, value in CPE_MARKET_KIND.items():
    codeList = instCpCodeMgr.GetStockListByMarket(value)
    for code in codeList:
        name = instCpCodeMgr.CodeToName(code)
        sectionKind = instCpCodeMgr.GetStockSectionKind(code)
        row = [code, name, key, sectionKind]
        rows.append(row)

print('모든 종목을 불러왔습니다')

stockitems = pd.DataFrame(data= rows, columns=['code','name', 'section','sectionKind'])
stockitems.loc[stockitems['sectionKind'] == 10, 'section'] =   'ETF'
stockitems.to_csv('stockitems.csv', index=False)

print('파일을 저장하였습니다.')

모든 종목을 불러왔습니다
파일을 저장하였습니다.


---
##### 모든 종목에 대한 차트데이터 요청
---

In [5]:
import win32com.client
import pandas as pd 
import time
import datetime

column_dailychart = ['code', 'section', 'date', 'open', 'high', 'low', 'close',
                     'vol', 'value', 'n_stock', 'agg_price', 'foreign_rate','agency_buy', 'agency_netbuy']

stockitems = pd.read_csv('stockitems.csv')

instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")

row = list(range(len(column_dailychart)))
rows = list()

instStockChart.SetInputValue(1, ord('1'))
instStockChart.SetInputValue(2, (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y%m%d"))
instStockChart.SetInputValue(3, '20180101')
instStockChart.SetInputValue(5, (0, 2, 3, 4, 5, 8, 9, 12, 13, 17, 20, 21))
instStockChart.SetInputValue(6, ord('D'))
instStockChart.SetInputValue(9, ord('1'))
    
for idx, stockitem in stockitems.iterrows():
    instStockChart.SetInputValue(0, stockitem['code'])
    
    # BlockRequest
    instStockChart.BlockRequest()

    # GetHeaderValue
    numData = instStockChart.GetHeaderValue(3)
    numField = instStockChart.GetHeaderValue(1)

    # GetDataValue
    for i in range(numData):
        row[0] =  stockitem['code']
        row[1] = stockitem['section']  # 코스피, 코스닥, ETF 여부
        row[2] = instStockChart.GetDataValue(0, i)  # 날짜
        row[3] = instStockChart.GetDataValue(1, i)  # 시가
        row[4] = instStockChart.GetDataValue(2, i)  # 고가
        row[5] = instStockChart.GetDataValue(3, i)  # 저가
        row[6] = instStockChart.GetDataValue(4, i)  # 종가
        row[7] = instStockChart.GetDataValue(5, i)  # 거래량
        row[8] = instStockChart.GetDataValue(6, i)  # 거래대금
        row[9] = instStockChart.GetDataValue(7, i)  # 상장주식수
        row[10] = instStockChart.GetDataValue(8, i)  # 시가총액
        row[11] = instStockChart.GetDataValue(9, i)  # 외국인 보율비율
        row[12] = instStockChart.GetDataValue(10, i)  # 기관순매수
        row[13] = instStockChart.GetDataValue(11, i)  # 기관누적순매수
        rows.append(list(row))

com_error: (-2147352567, '예외가 발생했습니다.', (0, None, 'U-CYBOS가 서버에 접속되어 있지 않습니다.', None, 0, -2147483638), None)

모든 종목을 불러왔습니다
파일을 저장하였습니다.


com_error: (-2147352567, '예외가 발생했습니다.', (0, None, 'U-CYBOS가 서버에 접속되어 있지 않습니다.', None, 0, -2147483638), None)

In [ ]:
import datetime

instStockChart.SetInputValue(1, ord('1'))
instStockChart.SetInputValue(2, (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y%m%d"))
instStockChart.SetInputValue(3, '20180101')

In [ ]:
nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

nCpCybos.GetLimitRemainCount(1) # 0: 주문관련 요청 / 1: 시세조회관련 요청

In [ ]:
import time

for idx, stockitem in stockitems.iterrows():

    remain_request_count =  nCpCybos.GetLimitRemainCount(1)
    print(stockitem['code'], stockitem['name'], '남은 요청 : ', remain_request_count)
    
    if remain_request_count == 0:
        print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')

        while True:
            time.sleep(2)
            remain_request_count =  nCpCybos.GetLimitRemainCount(1)
            if  remain_request_count > 0:
                print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                break
            print('대기 중...')
            
	...